In [2]:
using JuMP
using Gurobi
using Distributions
using Ipopt
using MLBase
using DataFrames
using DataFramesMeta
using MLDataUtils

In [3]:
myData1 = readtable("C:/Users/subha/Desktop/ML HW -3/magic04.csv",header=false);
myData2 = readtable("C:/Users/subha/Desktop/ML HW -3/parkinsons_updrs.csv",header=false);

In [4]:
function gamma1back0(y)
    newpredcol = []
    m = length(y)
    for i in 1:m
        if y[i]=="g"
            push!(newpredcol,1)
        else
            push!(newpredcol,0)
        end
    end
    newpredcol = Array{Float64}(newpredcol)
    return newpredcol
end

gamma1back0 (generic function with 1 method)

In [5]:
X = myData1[1:end-1];
Y = myData1[end];

In [6]:
function logistic_regression(X, y)
  n, p = size(X)

  # Convert y to (-1, +1)
  @assert(extrema(y) == (0, 1), "y must be all 0s and 1s")
  Y = y * 2 - 1
  @assert(extrema(Y) == (-1, 1))

  # Create the model and specify nonlinear solver
  # print_level=0 turns off solver output
  m = Model(solver=IpoptSolver(print_level=0))

  # Add variables
  @variable(m, β[1:p])
  @variable(m, β0)

  # Set nonlinear objective function with @NLobjective
  @NLobjective(m, Max, -sum(
    log(1 + exp(-Y[i] * (sum(X[i, j] * β[j] for j = 1:p) + β0)))
    for i = 1:n))

  # Solve the model and get the optimal solutions
  solve(m)
  return getvalue(β), getvalue(β0)
end

logistic_regression (generic function with 1 method)

In [7]:
function predict_logit(X, β, β0)
  [1.0 / (1.0 + e ^ -(dot(X[i, :], β) + β0)) for i = 1:size(X, 1)]
end

predict_logit (generic function with 1 method)

In [9]:
srand(1)

# Noted for Logistic Regression, there are no hyper parameters to test, so I do not have a validation set. 
(train_X, train_y), (test_X, test_y) = splitobs(shuffleobs((X, Y)), at=0.50)


train_X = Matrix(train_X)
test_X = Matrix(test_X)

train_Y = gamma1back0(train_y)
test_Y = gamma1back0(test_y)

β, β0 = logistic_regression(train_X, train_Y);
pred = predict_logit(test_X, β, β0);




******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



In [25]:
size(X)

count() in module IterTools at deprecated.jl:56 overwritten in module Iterators at deprecated.jl:56.

(19020, 10)

Number) in module IterTools at deprecated.jl:56 overwritten in module Iterators at deprecated.jl:56.


In [10]:
function labelpred(pred, threshold)
    Int.(pred .> threshold)
end

labelpred (generic function with 1 method)

In [13]:
best_percent = 0
best_threshold = Inf
for i in linspace(0,.95,50)
    label = labelpred(pred, i)
    percentcorrect = mean(label .== (test_Y))
    if percentcorrect > best_percent
        best_threshold = i
        best_percent = percentcorrect
    end
end
label = labelpred(pred,best_threshold)
println("The best percent correct is ", best_percent, " which occurs when the threshold is ", best_threshold)



The best percent correct is 0.7914826498422713 which occurs when the threshold is 0.5816326530612245


In [22]:
 test_Y = Array{Int64}(test_Y)

9510-element Array{Int64,1}:
 0
 1
 0
 1
 0
 1
 0
 1
 0
 1
 1
 0
 1
 ⋮
 1
 1
 1
 0
 1
 1
 1
 1
 1
 0
 1
 0

In [14]:
using AUC
auc(test_Y, pred)


0.8399763024025791

In [29]:
using MLBase

a = confusmat(2, test_Y + 1, label + 1) #create the confusion matrix here

# true neg , false posi
# false neg, true pos

#put a cost  and add weights on points within the logistic regression 
#


2×2 Array{Int64,2}:
 2202  1178
  805  5325

In [24]:

using ScikitLearn
# Load in the DecisionTreeClassifier method from ScikitLearn
@sk_import tree: DecisionTreeClassifier

INFO: Precompiling module ScikitLearn.

PyObject <class 'sklearn.tree.tree.DecisionTreeClassifier'>

In [40]:
function lossfun(y_true, y_pred, loss_value) 
    #y_true is the actual values
    #y_pred is the predictions
    #loss_value matrix
    a = confusmat(2, y_true + 1, y_pred + 1)
    VALUE  = a.*loss_value
    println("The loss function yields a score of ", VALUE[1,2], "due to ", a[1,2], " entries in the data which are false positives")
    println("The loss function yields a score of ", VALUE[2,1], "due to ", a[2,1], " entries in the data which are false negatives")
    return (sum(VALUE[2,1] + VALUE[1,2]))
end

lossfun (generic function with 1 method)

In [41]:
y_true = test_Y 
y_pred = label
loss_value = [[0,1],[20,0]]

    #y_true is the actual values
    #y_pred is the predictions
    #loss_value matrix
    a = confusmat(2, y_true + 1, y_pred + 1)
    VALUE  = a.*loss_value
    println("The loss function yields a score of ", VALUE[1,2], "due to ", a[1,2], " entries in the data which are false positives")
    println("The loss function yields a score of ", VALUE[2,1], "due to ", a[2,1], " entries in the data which are false negatives")
    return (sum(VALUE[2,1] + VALUE[1,2]))

The loss function yields a score of [0, 1178]due to 1178 entries in the data which are false positives
The loss function yields a score of [16100, 0]due to 805 entries in the data which are false negatives


17278

In [ ]:
Pkg.build("GraphViz")
using GraphViz
using PyCall
@pyimport sklearn.tree as sktree

function showtree(tree)
    g = Graph(sktree.export_graphviz(tree, out_file=nothing, filled=true, rounded=true))
    GraphViz.layout!(g; engine="dot")
    g
end

In [46]:
X = myData1[1:end-1];
Y = myData1[end];

# Set minbucket with `min_samples_leaf` argument, and maxdepth with `max_depth`.
srand(1)
(train_X, train_y), (test_X, test_y) = splitobs(shuffleobs((X, Y)), at=0.50)
(tr_X, tr_y), (vl_x, vl_y) = splitobs(shuffleobs((X, Y)), at=0.50)
# cross validation 
tr_X = Matrix(tr_X)
tr_y = gamma1back0(tr_y)
vl_x = Matrix(vl_x)
vl_y = gamma1back0(vl_y)
train_X = Matrix(train_X)
train_y = gamma1back0(train_y)

9510-element Array{Float64,1}:
 0.0
 1.0
 1.0
 0.0
 0.0
 0.0
 1.0
 1.0
 1.0
 0.0
 1.0
 0.0
 1.0
 ⋮  
 1.0
 0.0
 0.0
 1.0
 0.0
 1.0
 0.0
 1.0
 0.0
 1.0
 1.0
 1.0

In [47]:
tr_y = Array{Int64}(tr_y)
vl_y = Array{Int64}(vl_y)
train_y = Array{Int64}(train_y)



9510-element Array{Int64,1}:
 0
 1
 1
 0
 0
 0
 1
 1
 1
 0
 1
 0
 1
 ⋮
 1
 0
 0
 1
 0
 1
 0
 1
 0
 1
 1
 1

In [50]:


min_samples_leaf = 5
max_depth = 3
bestlosfunval = Inf
bestminbucket = Inf
bestdepth = Inf
tree = DecisionTreeClassifier(min_samples_leaf, max_depth)  
#         ScikitLearn.fit!(tree, tr_X, tr_y)
#         for min_samples_leaf in [5, 10, 50, 100, 200]
#             for max_depth in [1, 2, 3, 4, 5]
#                 label = ScikitLearn.predict(tree, vl_x)
#                 #showtree(tree)
#                 losfunval = lossfun(Array{Int64}(vl_y), label,[[0,1],[20,0]])
#                 if losfunval < bestlosfunval
#                     bestlosfunval = losfunval
#                     bestminbucket = min_samples_leaf
#                     bestdepth = max_depth
#                 end
#             end
#         end
# @show bestlosfunval, bestminbucket, bestdepth

# tuned_tree = DecisionTreeClassifier(bestminbucket, bestdepth)
#     ScikitLearn.fit!(tuned_tree, train_X, train_y)
#     label = ScikitLearn.predict(tuned_tree, test_X)
#     losfunval = lossfun(test_y, label, [0,1;20,1])
# println("For my best tree, my loss function value is", losfunval)


    

PyObject DecisionTreeClassifier(class_weight=None, criterion=5, max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter=3)